In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import seaborn as sns

import itertools
import re
import string
import pickle
import os

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import KFold

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud
from copy import deepcopy

from IPython.display import (
    Markdown as md,
    Latex,
    HTML,
)

from tqdm.auto import tqdm
import json

# set plot style
sns.set()

from google.colab import drive

drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 1: Load the cleaned CSV files
player_detail = pd.read_csv('/content/drive/MyDrive/nlp data/Data/player_detail_cleaned.csv')
transfer_history = pd.read_csv('/content/drive/MyDrive/nlp data/Data/transfer_history_cleaned.csv')
player_market_value_development = pd.read_csv('/content/drive/MyDrive/nlp data/Data/player_market_value_development_cleaned')

In [ ]:
player_detail.head()

,Unnamed: 0,Name,player_id,DOB,Age,Foot,Height_m,Nationality,Citizenship,Position,Other positions,Team,Joined,Contract expires
0,0,Thibaut Courtois,108390,1992-05-11,31.0,left,2.00,Belgium,['Belgium'],Goalkeeper,NaN,Real Madrid,2018-08-09,2026-06-30
1,1,Andriy Lunin,404839,1999-02-11,25.0,right,1.91,Ukraine,['Ukraine'],Goalkeeper,NaN,Real Madrid,2018-07-01,2025-06-30
2,2,Kepa Arrizabalaga,192279,1994-10-03,29.0,right,1.88,Spain,['Spain'],Goalkeeper,NaN,Real Madrid,2023-08-14,2024-06-30
3,3,Éder Militão,401530,1998-01-18,26.0,right,1.86,Brazil,"['Brazil', 'Spain']",Centre-Back,Right-Back,Real Madrid,2019-07-01,2028-06-30
4,4,David Alaba,59016,1992-06-24,31.0,left,1.80,Austria,['Austria'],Centre-Back,Left-Back,Real Madrid,2021-07-01,2026-06-30


In [ ]:
transfer_history.head()

,Unnamed: 0,from_clubName,from_latitude,from_longitude,to_clubName,to_latitude,to_longitude,url,futureTransfer,date,...,season,marketValue,fee,player_id,Total transfer fees,TM Market Value,Transfer Type,Transfer_Fee,main_from_clubName,main_to_clubName
0,0,Chelsea,51.477520,-0.201590,Real Madrid,40.452357,-3.686789,/thibaut-courtois/transfers/spieler/108390,0.0,2018-08-09,...,18/19,€65.00m,€35.00m,108390,45150000,65000000.0,Transfer,35000000.0,Chelsea,Real Madrid
1,1,Atlético Madrid,40.401730,-3.720635,Chelsea,51.477520,-0.201590,/thibaut-courtois/transfers/spieler/108390,0.0,2014-06-30,...,13/14,€25.00m,End of loan,108390,45150000,25000000.0,End of Loan,0.0,Atlético Madrid,Chelsea
2,2,Chelsea,51.477520,-0.201590,Atlético Madrid,40.401730,-3.720635,/thibaut-courtois/transfers/spieler/108390,0.0,2011-07-27,...,11/12,€4.00m,"Loan fee:<br /><i class=""normaler-text"">€1.20m...",108390,45150000,4000000.0,Loan,1200000.0,Chelsea,Atlético Madrid
3,3,KRC Genk,51.005026,5.533332,Chelsea,51.477520,-0.201590,/thibaut-courtois/transfers/spieler/108390,0.0,2011-07-26,...,11/12,€4.00m,€8.95m,108390,45150000,4000000.0,Transfer,8950000.0,KRC Genk,Chelsea
4,4,KRC Genk U19,51.005026,5.533332,KRC Genk,51.005026,5.533332,/thibaut-courtois/transfers/spieler/108390,0.0,2009-07-01,...,09/10,€50k,-,108390,45150000,50000.0,Transfer,0.0,KRC Genk,KRC Genk


In [ ]:
player_market_value_development.head()

,Unnamed: 0,mw,datum_mw,verein,age,player_id,Highest_market_value,highest_date,last_change
0,0,50000.0,2009-05-27,KRC Genk U19,17,108390,75000000.0,2020-10-08,2024-03-21
1,1,150000.0,2010-07-28,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
2,2,1000000.0,2010-10-06,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
3,3,2000000.0,2010-12-28,KRC Genk,18,108390,75000000.0,2020-10-08,2024-03-21
4,4,3000000.0,2011-05-24,KRC Genk,19,108390,75000000.0,2020-10-08,2024-03-21


In [ ]:
# Step 2: Calculate Number of Transfers
transfer_count = transfer_history.groupby('player_id')['to_clubName'].nunique().reset_index()
transfer_count.columns = ['player_id', 'number_of_transfers']
player_detail = pd.merge(player_detail, transfer_count, on='player_id', how='left')
player_detail['number_of_transfers'].fillna(0, inplace=True)

# Calculate Total Transfer Fees
total_transfer_fees = transfer_history.groupby('player_id')['Transfer_Fee'].sum().reset_index()
total_transfer_fees.columns = ['player_id', 'total_transfer_fees']
player_detail = pd.merge(player_detail, total_transfer_fees, on='player_id', how='left')
player_detail['total_transfer_fees'].fillna(0, inplace=True)

# Step 3: Extract Market Values and Highest Market Value
latest_market_value = player_market_value_development.sort_values(by=['player_id', 'datum_mw']).groupby('player_id').tail(1)[['player_id', 'mw']].reset_index(drop=True)
latest_market_value.columns = ['player_id', 'latest_market_value']
highest_market_value = player_market_value_development.groupby('player_id')['Highest_market_value'].max().reset_index()
highest_market_value.columns = ['player_id', 'highest_market_value']
player_detail = pd.merge(player_detail, latest_market_value, on='player_id', how='left')
player_detail = pd.merge(player_detail, highest_market_value, on='player_id', how='left')
player_detail['latest_market_value'].fillna(0, inplace=True)
player_detail['highest_market_value'].fillna(0, inplace=True)

# Step 4: Extract Latest Transfer Fee and Transfer Type
# Convert date column to datetime for accurate sorting
transfer_history['date'] = pd.to_datetime(transfer_history['date'])

# Sort by date and then get the latest transfer info for each player
latest_transfer_info = transfer_history.sort_values(by=['player_id', 'date']).groupby('player_id').tail(1)[['player_id', 'Transfer_Fee', 'Transfer Type']]

# Rename columns for clarity
latest_transfer_info.columns = ['player_id', 'latest_transfer_fee', 'latest_transfer_type']

# Merge latest transfer info into player_detail
player_detail = pd.merge(player_detail, latest_transfer_info, on='player_id', how='left')

# Fill NaN values in 'latest_transfer_fee' and 'latest_transfer_type' with 0 or 'Unknown' for players with no transfer data
player_detail['latest_transfer_fee'].fillna(0, inplace=True)
player_detail['latest_transfer_type'].fillna('Unknown', inplace=True)

# Print the first few rows to verify the result
print(player_detail.head())

# Save the modified player_detail DataFrame to a CSV file (optional)
player_detail.to_csv('/content/drive/MyDrive/nlp data/Data/player_detail_enhanced.csv', index=False)

   Unnamed: 0               Name  player_id         DOB   Age   Foot  \
0           0   Thibaut Courtois     108390  1992-05-11  31.0   left   
1           1       Andriy Lunin     404839  1999-02-11  25.0  right   
2           2  Kepa Arrizabalaga     192279  1994-10-03  29.0  right   
3           3       Éder Militão     401530  1998-01-18  26.0  right   
4           4        David Alaba      59016  1992-06-24  31.0   left   

   Height_m Nationality          Citizenship     Position Other positions  \
0      2.00     Belgium          ['Belgium']   Goalkeeper             NaN   
1      1.91     Ukraine          ['Ukraine']   Goalkeeper             NaN   
2      1.88       Spain            ['Spain']   Goalkeeper             NaN   
3      1.86      Brazil  ['Brazil', 'Spain']  Centre-Back      Right-Back   
4      1.80     Austria          ['Austria']  Centre-Back       Left-Back   

          Team      Joined Contract expires  number_of_transfers  \
0  Real Madrid  2018-08-09       202

In [ ]:
player_detail.head()

,Unnamed: 0,Name,player_id,DOB,Age,Foot,Height_m,Nationality,Citizenship,Position,Other positions,Team,Joined,Contract expires,number_of_transfers,total_transfer_fees,latest_market_value,highest_market_value,latest_transfer_fee,latest_transfer_type
0,0,Thibaut Courtois,108390,1992-05-11,31.0,left,2.00,Belgium,['Belgium'],Goalkeeper,NaN,Real Madrid,2018-08-09,2026-06-30,6.0,45150000.0,30000000.0,75000000.0,35000000.0,Transfer
1,1,Andriy Lunin,404839,1999-02-11,25.0,right,1.91,Ukraine,['Ukraine'],Goalkeeper,NaN,Real Madrid,2018-07-01,2025-06-30,7.0,8500000.0,16000000.0,16000000.0,0.0,End of Loan
2,2,Kepa Arrizabalaga,192279,1994-10-03,29.0,right,1.88,Spain,['Spain'],Goalkeeper,NaN,Real Madrid,2023-08-14,2024-06-30,9.0,81000000.0,12000000.0,60000000.0,0.0,End of Loan
3,3,Éder Militão,401530,1998-01-18,26.0,right,1.86,Brazil,"['Brazil', 'Spain']",Centre-Back,Right-Back,Real Madrid,2019-07-01,2028-06-30,4.0,57000000.0,70000000.0,70000000.0,50000000.0,Transfer
4,4,David Alaba,59016,1992-06-24,31.0,left,1.80,Austria,['Austria'],Centre-Back,Left-Back,Real Madrid,2021-07-01,2026-06-30,9.0,150000.0,25000000.0,65000000.0,0.0,Transfer
